how to extend single document agent to handle multiple documents, with increasing degrees of complexity

In [3]:
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
#download papers

urls =[
    "https://openreview.net/pdf?id=VytmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf"
]

In [14]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex,SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

In [17]:
def get_doc_tools(
    file_path:str,
    name:str,
) -> str:
    """Get vector query and summary query tools from a document."""
    
    # load documents
    documents = SimpleDirectoryReader(input_files = [file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)
    
    def vector_query(
        query:str,
        page_numbers: optional[list[str]] = None
    ) -> str:
        """Use to answer questions over MetaGPT paper.
        
        Useful if you have specific questions over MetaGPT paper.
        Always leave page_numbers as None unless there is specific page you want to search for.
        
        Args:
            query (str): the string query to be embedded
            page_numbers (optional[list[str]]): filter by set of pages. Leave as None 
                            if we want to perform a vector search
                            over all pages. Otherwise, filter by the set of specified pages.
        """
        
        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value":p} for p in page_numbers
        ]
        
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response
    
    vector_query_tool = FunctionTool.from_defaults(
        name = f"Vector_tool_{name}",
        fn=vector_query
    )
    
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode = "tree_summarize",
        use_async = True,
    )
    
    summary_tool = QueryEngineTool.from_defaults(
        name=f"Summary_tool_{name}",
        query_engine=summary_query_engine,
        description = (
            "Use only if you want to get a holistic summary of MetaGPT."
            "Do not use if you have specific questions over MetaGPT"
        ),
    )
    
    return vector_query_tool, summary_tool
    
    
        

In [ ]:
#convert each paper into  tool
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]


Getting tools for paper: metagpt.pdf


ValueError: File metagpt.pdf does not exist.

In [ ]:
documents= SimpleDirectoryReader(input_files = [])